# Fully Connected Neural Network for MNIST

## Requirements

### Python-Modules

In [ ]:
# third party
import numpy as np
# mnist data
from deep_teaching_commons.data.fundamentals.mnist import Mnist
# tensorflow
import tensorflow as tf

### Data

In [2]:
# create mnist loader from deep_teaching_commons
mnist_loader = Mnist(data_dir='data')

# load all data, labels are one-hot-encoded, images are flatten and pixel squashed between [0,1]
train_images, train_labels, test_images, test_labels = mnist_loader.get_all_data(flatten=False, one_hot_enc=True, normalized=True)
print(train_images.shape, train_labels.shape)

# reshape to match generel framework architecture
train_images, test_images = train_images.reshape(60000, 28, 28, 1), test_images.reshape(10000, 28, 28, 1)
print(train_images.shape, train_labels.shape)

# shuffle training data
shuffle_index = np.random.permutation(60000)
train_images, train_labels = train_images[shuffle_index], train_labels[shuffle_index]

auto download is active, attempting download
mnist data directory already exists, download aborted
(60000, 28, 28) (60000, 10)
(60000, 28, 28, 1) (60000, 10)


## Building the Neural Network
For a better understanding of neural networks you will start to implement your own framework. The given notebook explaines some core functions and concetps of the framework, so you all have the same starting point. The  Pipeline will be: 

**define a model architecture -> construct a neural network from the model -> define a evaluation citeria -> optimize the network**

### Creating a custom architecture
To create a custom model you have to define layers and activation functions that can be used to do so. Layers and activation functions are modeled as objects. Each object that you want to use has to implement a `forward` that is used by the `NeuralNetwork` class. Additionally the `self.params` attribute is mandatory to meet the specification of the `NeuralNetwork` class. It is used to store all learnable parameters that you need for the optimization algorithm. We implement our neural network so that we can use the objects as building blocks and stack them up to create a custom model. 

#### Layers  Class
The file `layer.py` contains implementations of neural network layers and regularization techniques that can be inserted as layers into the architecture.

In [3]:
class Flatten():
    ''' Flatten layer used to reshape inputs into vector representation
    
    Layer should be used in the forward pass before a dense layer to 
    transform a given tensor into a vector. 
    '''
    def __init__(self):
        self.params = []

    def forward(self, X):
        ''' Reshapes a n-dim representation into a vector 
            by preserving the number of input rows.
        
        Args:
            X: Images set
    
        Returns:
            X_: Matrix with images in a flatten represenation
            
        Examples:
            [10000,[1,28,28]] -> [10000,784]
        '''
        return tf.reshape(X, [-1, X.shape[1] * X.shape[2] * X.shape[3]])

In [4]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
XX = Flatten().forward(X)

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    test = sess.run(XX, feed_dict={X:train_images})
    print(test.shape)

(60000, 784)


In [6]:
class FullyConnected():
    ''' Fully connected layer implemtenting linear function hypothesis 
        in the forward pass and its derivation in the backward pass.
    '''
    def __init__(self, in_size, out_size, stddev=0.1):
        ''' Initilize all learning parameters in the layer
        
        Weights will be initilized with modified Xavier initialization.
        Biases will be initilized with zero. 
        '''
        self.W = tf.Variable(tf.truncated_normal([in_size, out_size], stddev=stddev))
        self.b = tf.Variable(tf.ones([out_size])/10)
        self.params = [self.W, self.b]

    def forward(self, X):
        ''' Linear combiationn of images, weights and bias terms
            
        Args:
            X: Matrix of images (flatten represenation)
    
        Returns:
            out: Sum of X*W+b  
        '''
        return tf.matmul(X, self.W) + self.b

In [8]:
Z = FullyConnected(784, 200).forward(XX)

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    test = sess.run(Z, feed_dict={X:train_images})
    print(test.shape)

(60000, 200)


### Activation Functions
The file `activation_func.py` contains implementations of activation functions you can use as a none linearity in your network. The functions work on the basis of matrix operations and not discret values, so that these can also be inserted as a layer. As an example the ReLU function is implemented:

$$
f ( x ) = \left\{ \begin{array} { l l } { x } & { \text { if } x > 0 } \\ { 0 } & { \text { otherwise } } \end{array} \right.
$$

In [9]:
class ReLU():
    ''' Implements activation function rectified linear unit (ReLU) 
    
    ReLU activation function is defined as the positive part of 
    its argument. Todo: insert arxiv paper reference
    '''
    def __init__(self):
        self.params = []

    def forward(self, X):
        ''' In the forward pass return the identity for x < 0
        
        Safe input for backprop and forward all values that are above 0.
        '''
        return tf.nn.relu(X)

In [11]:
A = ReLU().forward(Z)

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    test = sess.run(A, feed_dict={X:train_images})
    print(test.shape)
    print(np.min(test))

(60000, 200)
0.0


## NeuralNetwork class

In [ ]:
class NeuralNetwork:
    ''' Creates a neural network from a given layer architecture 
    
    This class is suited for fully connected network and
    convolutional neural network architectures. It connects 
    the layers and passes the data from one end to another.
    '''
    def __init__(self, layers, score_func=None):
        ''' Setup a global parameter list and initilize a
            score function that is used for predictions.
        
        Args:
            layer: neural network architecture based on layer and activation function objects
            score_func: function that is used as classifier on the output
        '''
        self.layers = layers
        self.params = []
        for layer in self.layers:
            self.params.append(layer.params)
        self.score_func = score_func

    def forward(self, X):
        ''' Pass input X through all layers in the network 
        '''
        for layer in self.layers:
            X = layer.forward(X)
        return X

    def backward(self, dout):
        grads = []
        ''' Backprop through the network and keep a list of the gradients
            from each layer.
        '''
        for layer in reversed(self.layers):
            dout, grad = layer.backward(dout)
            grads.append(grad)
        return grads

    def predict(self, X):
        ''' Run a forward pass and use the score function to classify 
            the output.
        '''
        X = self.forward(X)
        return np.argmax(self.score_func(X), axis=1)

## Loss Function
Implementations of loss functions can be found in `loss_func.py`. A loss function object defines the criteria your network is evaluated during the optimization process and also contains score functions that can be used as classification criteria for predictions with the final model. Therefore it is necessary to create a loss function object and provide to the optimization algorithm.

In [ ]:
class LossCriteria():
    ''' Implements different typs of loss and score functions for neural networks
    
    Todo:
        - Implement init that defines score and loss function 
    '''
    def softmax(X):
        ''' Numeric stable calculation of softmax
        '''
        ############################################
        #TODO: Implement the Softmax function      #
        ############################################
        raise NotImplementedError("You should implement this!")
        ############################################
        #             END OF YOUR CODE             #
        ############################################

    def cross_entropy_softmax(X, y):
        ''' Computes loss and prepares dout for backprop 

        https://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative/
        '''
        m = y.shape[0]
        p = LossCriteria.softmax(X)
        log_likelihood = -np.log(p[range(m), y])
        loss = np.sum(log_likelihood) / m
        dout = p.copy()
        dout[range(m), y] -= 1
        return loss, dout

## Optimization with SGD
The file `optimizer.py` contains implementations of optimization algorithms. Your optimizer needs your custom `network`, `data` and `loss function` and some additional hyperparameter as arguments to optimzie your model. 

In [ ]:
class Optimizer():   
    def get_minibatches(X, y, batch_size):
        ''' Decomposes data set into small subsets (batch)
        '''
        m = X.shape[0]
        batches = []
        for i in range(0, m, batch_size):
            X_batch = X[i:i + batch_size, :, :, :]
            y_batch = y[i:i + batch_size, ]
            batches.append((X_batch, y_batch))
        return batches    

    def sgd(network, X_train, y_train, loss_function, batch_size=32, epoch=100, learning_rate=0.001, X_test=None, y_test=None, verbose=None):
        ''' Optimize a given network with stochastic gradient descent 
        '''
        minibatches = Optimizer.get_minibatches(X_train, y_train, batch_size)
        for i in range(epoch):
            loss = 0
            if verbose:
                print('Epoch',i + 1)
            for X_mini, y_mini in minibatches:
                # calculate loss and derivation of the last layer
                loss, dout = loss_function(network.forward(X_mini), y_mini)
                # calculate gradients via backpropagation
                grads = network.backward(dout)
                # run vanilla sgd update for all learnable parameters in self.params
                for param, grad in zip(network.params, reversed(grads)):
                    for i in range(len(grad)):
                        param[i] += - learning_rate * grad[i]
            if verbose:
                train_acc = np.mean(y_train == network.predict(X_train))
                test_acc = np.mean(y_test == network.predict(X_test))                                
                print("Loss = {0} :: Training = {1} :: Test = {2}".format(loss, train_acc, test_acc))
        return network

# Put it all together
Now you have parts together to create and train a fully connected neural network. First, you have to define an individual network architecture by flatten the input and stacking fully connected layer with activation functions. Your custom architecture is given to a `NeuralNetwork` object that handles the inter-layer communication during the forward and backward pass. Finally, you optimize the model with a chosen algorithm, here stochastic gradient descent. That kind of pipeline is similar to the one you would create with a framework like Tensorflow or PyTorch.

In [ ]:
# design a three hidden layer architecture with Dense-Layer
# and ReLU as activation function
def fcn_mnist():
    flat = Flatten()
    hidden_01 = FullyConnected(784, 500)
    relu_01 = ReLU()
    hidden_02 = FullyConnected(500, 200)
    relu_02 = ReLU()
    hidden_03 = FullyConnected(200, 100)
    relu_03 = ReLU()
    ouput = FullyConnected(100, 10)
    return [flat, hidden_01, relu_01, hidden_02, relu_02, hidden_03, relu_03, ouput]

# create a neural network on specified architecture with softmax as score function
fcn = NeuralNetwork(fcn_mnist(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
fcn = Optimizer.sgd(fcn, train_images, train_labels, LossCriteria.cross_entropy_softmax, batch_size=64, epoch=10, learning_rate=0.01, X_test=test_images, y_test=test_labels, verbose=True)

# ToDo: Create your own Architecture on your own data set

Your main goal is to explore the given implementation, to perform experiments with different model combinations and to document your observations. So do the following:
  * Write a data loader for a different image dataset, e.g., [CIFAR](https://www.cs.toronto.edu/~kriz/cifar.html) or [Labeled Faces in the Wild](http://vis-www.cs.umass.edu/lfw/). Feel free to search a dataset you like to classify.
  * Create and train different architectures using thr given implementation.
  * Compare the results and document your observations.